In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import tensorflow as tf
from vit_keras import vit
from skimage import transform

C:\Users\Farid.Guliyev\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
INPUT_IMAGES = "input/"
# TARGET_IMAGES = "Target/target "

TRAIN_LIMIT_SIZE = 15000
# VALIDATION_LIMIT_SIZE = 17500
TEST_LIMIT_SIZE = 20000
BATCH_SIZE = 32

STEPS_SIZE = TRAIN_LIMIT_SIZE // BATCH_SIZE

THRESHOLD_CLEANSING = 30

In [3]:
RA_RANGE = (-180,180)
DEC_RANGE = (-90,90)
THETA_RANGE = (0,360)

In [4]:
import pandas as pd

# importing
df_orientation = pd.read_csv("orientation_data.csv")
df_orientation = df_orientation.drop("ID",axis = 1)
df_orientation = df_orientation.drop("FOV",axis = 1)

# minmax normalization
df_orientation["RA"] = (df_orientation.iloc[:,0] - RA_RANGE[0]) / (RA_RANGE[1] - RA_RANGE[0])
df_orientation["DEC"] = (df_orientation.iloc[:,1] - DEC_RANGE[0]) / (DEC_RANGE[1] - DEC_RANGE[0])
df_orientation["THETA"] = (df_orientation.iloc[:,2] - THETA_RANGE[0]) / (THETA_RANGE[1] - THETA_RANGE[0])
df_orientation.head()

,RA,DEC,THETA
0,0.208573,0.278189,0.759450
1,0.208573,0.278189,0.759450
2,0.653541,0.525090,0.636799
3,0.134132,0.462668,0.167021
4,0.680660,0.604051,0.447248


In [5]:
import os

image_shape = (512, 512,3)

def load_images(image_name):
    img = cv2.imread(image_name)
    img = transform.resize(img,image_shape)
    return img

def image_generator():
    while True:
        for i in range(0,STEPS_SIZE):
            batch_x_train = []
            for j in range(i*BATCH_SIZE,(i+1)*BATCH_SIZE):
                batch_x_train.append(load_images(INPUT_IMAGES+str(j+1)+".png"))
            yield np.array(batch_x_train), df_orientation.iloc[(i*BATCH_SIZE):((i+1) * BATCH_SIZE),:].values

## Building the Model

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.activations import sigmoid

from tensorflow.keras.models import Model

In [7]:
from vit_keras import vit

In [33]:
base_model = vit.vit_l16(image_size = (512,512), activation = "sigmoid", include_top = False, 
                         pretrained=True, pretrained_top=False, weights='imagenet21k+imagenet2012' )

In [34]:
model = base_model

In [35]:
for layer in model.layers:
    layer.trainable = False

In [37]:
# Add output layers
x = base_model.output
x = Dense(256, activation='relu')(x)
output_layer = Dense(3, activation='sigmoid')(x)  # 3 sigmoid nodes for RA, DEC, and THETA

In [38]:
model = Model(inputs=base_model.input, outputs = output_layer)

In [39]:
from keras.optimizers import Adam
model.compile(optimizer="adam", loss='mse')

In [40]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 32, 32, 1024)      787456    
                                                                 
 reshape_4 (Reshape)         (None, 1024, 1024)        0         
                                                                 
 class_token (ClassToken)    (None, 1025, 1024)        1024      
                                                                 
 Transformer/posembed_input  (None, 1025, 1024)        1049600   
  (AddPositionEmbs)                                              
                                                                 
 Transformer/encoderblock_0  ((None, 1025, 1024),      12596224  
  (TransformerBlock)          (None, 16, None, None)       

## Model training

In [44]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("vit_model", monitor='loss')

In [45]:
train_generator = image_generator()

In [46]:
hist = model.fit(
    train_generator,
    steps_per_epoch = STEPS_SIZE,
    epochs = 100,
    callbacks = [checkpoint]
)

Epoch 1/100
468/468 [==============================] - ETA: 0s - loss: 0.0579  INFO:tensorflow:Assets written to: vit_model\assets


INFO:tensorflow:Assets written to: vit_model\assets


468/468 [==============================] - 36285s 78s/step - loss: 0.0579
Epoch 2/100
468/468 [==============================] - ETA: 0s - loss: 0.0499  INFO:tensorflow:Assets written to: vit_model\assets


INFO:tensorflow:Assets written to: vit_model\assets


468/468 [==============================] - 36804s 79s/step - loss: 0.0499
Epoch 3/100
468/468 [==============================] - ETA: 0s - loss: 0.0480  INFO:tensorflow:Assets written to: vit_model\assets


...

INFO:tensorflow:Assets written to: vit_model\assets


468/468 [==============================] - 47563s 102s/step - loss: 0.0469
Epoch 5/100
468/468 [==============================] - ETA: 0s - loss: 0.0460  INFO:tensorflow:Assets written to: vit_model\assets


INFO:tensorflow:Assets written to: vit_model\assets


468/468 [==============================] - 47009s 100s/step - loss: 0.0460
Epoch 6/100
376/468 [=======================>......] - ETA: 2:46:27 - loss: 0.0455

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/Transformer/encoderblock_2/MultiHeadDotProductAttention_1/MatMul' defined at (most recent call last):
    File "C:\Users\Farid.Guliyev\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Farid.Guliyev\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Farid.Guliyev\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Farid.Guliyev\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Farid.Guliyev\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Farid.Guliyev\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\U